In [1]:
from hashlib import sha256
import json
from flask import Flask, request
import requests

class Block:
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce

    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

In [5]:
import time

class Blockchain: 
    def __init__(self):
        self.unconfirmed_transactions = ["Ty pays Mr Brent 1000 BTC"]
        self.chain = []
        self.difficulty = 5
        self.create_genesis_block()
 
    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
    @property
    def last_block(self):
        return self.chain[-1]
    def proof_of_work(self, block):
        block.nonce = 0
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash
    def mine(self):
        if not self.unconfirmed_transactions:
            return False
 
        last_block = self.last_block
 
        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)
 
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)
        self.unconfirmed_transactions = ["Ty pays Mr Brent 1000 BTC"]
        return new_block.index
    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0' * self.difficulty) and
                block_hash == block.compute_hash())
    def add_block(self, block, proof):
        previous_hash = self.last_block.hash
        if previous_hash != block.previous_hash:
            return False
        if not self.is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)
        return True
    def add_new_transaction(self, transaction):
            self.unconfirmed_transactions.append(transaction)

In [6]:
CoolChain = Blockchain()
while len(CoolChain.chain) < 10:
    
    print(CoolChain.chain[-1].hash)

    CoolChain.mine()

021f704d6a5ec852f2cf0722dae2d14451934e44e53c1021e1767a88b297cfa8
000006fb0a82e2b80cd586e5448ecd4eb80945ff203fed3766c885061a1a2066
00000b6e0eebc3bec71f015a4d9b0355aba5bf49156ae46ea0c652a4152a8f50
0000092293cfb0203dab0db62df1da1e054a8f69087f88d70cb9be06697f4830
000005ef78628df27a7a729278768221cd2f626b3f6becab8dcefc48dcc45a8e
000000f024e8537222bcfb80da22bf128d92b0dc34717bf49beb7f71745a3cac
00000201d50bded128bc8820564d587f4c373a293956d82e40280ad01a47e138
000009dd746b44a5bf3d10cfbbe7d651b9abe1877202f3107a11146888df546d
00000b9beca63872c1943f08efb33f29a0307a1ec690bc10c6ebe9dae5e3bc82
